In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
data_dir = './data/raw'

train_datagen = ImageDataGenerator(
    validation_split = .20, 
    rescale= 1.0/255,
    rotation_range = 40,
    width_shift_range = .2,
    height_shift_range = .2,
    shear_range = .2,
    zoom_range=.2,
    horizontal_flip = True,
    fill_mode="nearest",
)

In [3]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# format fruit labels
class_labels = {idx: name.replace(' fruit', '') for name, idx in train_generator.class_indices.items() }

print(class_labels)

Found 288 images belonging to 9 classes.
Found 71 images belonging to 9 classes.
{0: 'apple', 1: 'banana', 2: 'cherry', 3: 'chickoo', 4: 'grapes', 5: 'kiwi', 6: 'mango', 7: 'orange', 8: 'strawberry'}


In [4]:
# Define a simple CNN model for classification
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')  # 9 output classes for fruit categories
])

c:\Users\antho\Coding\repos\fruit-classifier\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',      # Use 'binary_crossentropy' for binary classification
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=(train_generator.samples // train_generator.batch_size) + 1,
    epochs=30,                         # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=(validation_generator.samples // validation_generator.batch_size) + 1
)


c:\Users\antho\Coding\repos\fruit-classifier\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
18/19 ━━━━━━━━━━━━━━━━━━━━ 0s 890ms/step - accuracy: 0.1925 - loss: 5.5510

C:\Users\antho\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
c:\Users\antho\Coding\repos\fruit-classifier\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.1926 - loss: 5.4496 - val_accuracy: 0.2958 - val_loss: 1.9247
Epoch 2/30
17/19 ━━━━━━━━━━━━━━━━━━━━ 1s 910ms/step - accuracy: 0.2402 - loss: 1.8791